# Sheep Counting Using Object Detection and ByteTrack 
## **Polygone zone**

This notebook is based on the occupancy analysis [tutorial](https://supervision.roboflow.com/develop/notebooks/occupancy_analytics/#install-relevant-packages) from supervision.

## 1. Model Loading and Initialization:

In [ ]:
import os

import numpy as np
import supervision as sv
from ultralytics import YOLO

### Define directories

In [2]:
HOME = os.getcwd()
print(HOME)

c:\Users\helary_l\Documents\ICAERUS\code\scripts_utiles\track


In [ ]:
VIDEO_PATH = "../../data/Videos"
print(VIDEO_PATH)

c:/Users/helary_l/Documents/ICAERUS/Images_drone/Sheep_drone_images/travail/cropped_videos


### Load video file

In [ ]:
FILE = "sheep_crossing.MP4"
SOURCE_VIDEO_PATH = f"{VIDEO_PATH}/{FILE}"

In [5]:
print(SOURCE_VIDEO_PATH)

c:/Users/helary_l/Documents/ICAERUS/Images_drone/Sheep_drone_images/travail/cropped_videos/crop_Passage_brebis_portique_2_30fps.MP4


In [ ]:
video_info = sv.VideoInfo.from_video_path(SOURCE_VIDEO_PATH)
print(video_info)

### Define export folder

In [ ]:
EXPORT = 'path/to/export/folder'

### Load pre-trained model

In [ ]:
# choose the model to use
SHEEP_MODEL = "../../Models/02_sheep_detection_v1/sheep_v1.pt"
print(SHEEP_MODEL)

In [ ]:
# load YOLO model 
model = YOLO(SHEEP_MODEL)
model.fuse()

Model summary (fused): 218 layers, 25840339 parameters, 0 gradients


## 2. Video Processing and Object Detection

In [ ]:
# detection of objects on the video and recording of a new video with object predictions according to the YOLO model used
%cd {HOME}
!yolo task=detect mode=predict model="path/to/detector.pt" conf=0.50 source={SOURCE_VIDEO_PATH}

c:\Users\helary_l\Documents\ICAERUS\code\scripts_utiles\track
Ultralytics YOLOv8.2.8 🚀 Python-3.11.9 torch-2.3.0 CPU (13th Gen Intel Core(TM) i5-1335U)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients

video 1/1 (frame 1/600) c:\Users\helary_l\Documents\ICAERUS\Images_drone\Sheep_drone_images\travail\cropped_videos\crop_23.11.23-12_30fps.MP4: 960x1280 (no detections), 949.2ms
video 1/1 (frame 2/600) c:\Users\helary_l\Documents\ICAERUS\Images_drone\Sheep_drone_images\travail\cropped_videos\crop_23.11.23-12_30fps.MP4: 960x1280 (no detections), 1107.2ms
video 1/1 (frame 3/600) c:\Users\helary_l\Documents\ICAERUS\Images_drone\Sheep_drone_images\travail\cropped_videos\crop_23.11.23-12_30fps.MP4: 960x1280 (no detections), 1022.5ms
video 1/1 (frame 4/600) c:\Users\helary_l\Documents\ICAERUS\Images_drone\Sheep_drone_images\travail\cropped_videos\crop_23.11.23-12_30fps.MP4: 960x1280 (no detections), 1249.2ms
video 1/1 (frame 5/600) c:\Users\helary_l\Documents\ICAERUS\Images_d

## 3. Integrating ByteTrack for Object Tracking and Sheep Counting Using a Virtual Polygon Zone


### Define the position of the polygon zone

In [ ]:
# optional : create a directory to save the video frames
# helps to define the polygon zone at a later stage
from PIL import Image

FRAMES_DIR = f"{VIDEO_PATH}/frames_{SOURCE_VIDEO_PATH[-15:-4]}"
os.mkdir(FRAMES_DIR)
print(FRAMES_DIR)

frames_generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

for i, frame in enumerate(frames_generator):
  img = Image.fromarray(frame, mode = "RGB")
  img.save(f"{FRAMES_DIR}/{i}.jpg")

print(f"Saved frames to {FRAMES_DIR}")

FileExistsError: [WinError 183] Impossible de créer un fichier déjà existant: 'c:/Users/helary_l/Documents/ICAERUS/Images_drone/Sheep_drone_images/travail/cropped_videos/frames_que_2_30fps'

#### Configurate zone

[draw points to create polygon coordinates](https://roboflow.github.io/polygonzone/)

In [ ]:
# copy-paste polygon coordinates from supervision tool above. 
zone = {
        'name' : 'Counting',
        'polygon' : np.array([
            [542, 736], [463, 920], [1039, 924], [898, 742]
            ]),
        'max' : 20
    }

'\nscene = np.zeros((100, 100, 3), dtype=np.uint8)\ntext_anchor = sv.Point(x=50, y=50)\n'

Create a video generator for our sample input file and display its first frame on the screen

In [ ]:
generator = sv.get_video_frames_generator(SOURCE_VIDEO_PATH)

In [ ]:
frame = next(generator)

sv.plot_image(frame, (12, 12))

### Define tracker parameters

In [ ]:
tracker = sv.ByteTrack(
    track_buffer=5,
    track_thresh=0.6, # default = 0.6
    match_thresh=0.8, # default = 0.8
    frame_rate=video_info.fps
)

### Define callback function

In [ ]:
def callback(frame: np.ndarray) -> sv.Detections:
    result = model(frame)[0]
    detection_model = sv.Detections.from_ultralytics(result)
    detections = tracker.update_with_detections(detection_model)
    return detections # detections

#### Improve vizualization


In [ ]:
color = sv.ColorPalette.DEFAULT
color_type = sv.ColorLookup.TRACK

# add annotators
bounding_box_annotator = sv.BoundingBoxAnnotator(color=color,color_lookup=color_type)
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator(thickness=4)
def setup_zones(frame_wh):
  zone['unique_id'] = set() 
  zone['numero_id'] = []
  zone['PolygonZone'] = sv.PolygonZone(
      polygon=zone['polygon'],
      frame_resolution_wh=frame_wh
  )
  zone['PolygonZoneAnnotator'] = sv.PolygonZoneAnnotator(
    zone=zone['PolygonZone'],
    color=sv.Color.WHITE,
    thickness=4
)

def process_frame(frame):
    detections = callback(frame)
    # objets détectés dans la zone oui/non
    zone_presence = zone['PolygonZone'].trigger(detections)
    # extract id of tracking objects in polygon zone 
    zone_present_idxs = [idx for idx, present in enumerate(zone_presence) if present]
    zone_present = detections[zone_present_idxs]
    zone['unique_id'] = zone['unique_id'].union(set(zone_present.tracker_id))
    print('unique id',zone['unique_id'])
    zone['numero_id'].append(zone_present.tracker_id)

    # define labels to show on each bounding box
    labels = [
        f"#{tracker_id} {confidence:0.2f}"
        for tracker_id, confidence
        in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]

    # display objects in zone (detections = detections to display all detected objects)
    annotated_frame = bounding_box_annotator.annotate(
      scene=frame.copy(), 
      detections=zone_present
      )

    annotated_frame = trace_annotator.annotate(
        scene=annotated_frame,
        detections=zone_present)

    annotated_frame = label_annotator.annotate(
        scene=annotated_frame,
        detections=zone_present,
        labels=labels)
    
    annotated_frame = label_annotator.annotate(
    scene=annotated_frame,
    detections=zone_present,
    labels=labels)
    
    # total number of objects detected displayed in zone
    frame_annotees = zone['PolygonZoneAnnotator'].annotate(
        scene = annotated_frame,
        label = f"{zone['name']}: {len(zone['unique_id'])}"
    )

    return frame_annotees

#### on a single image

In [ ]:
image = cv2.imread(f"{FRAMES_DIR}/10.jpg")
image_wh = (image.shape[1],image.shape[0])
setup_zones(image_wh)

annotated_image = process_frame(image)
sv.plot_image(annotated_image)

NameError: name 'FRAMES_DIR' is not defined

#### Process video

In [ ]:
# choose the output repository and output file name
MAIN_OUTPUT_PATH = EXPORT + f"/name_of_output_file"
print(MAIN_OUTPUT_PATH)

c:/Users/helary_l/Documents/ICAERUS/Images_drone/Sheep_drone_images/travail/analyses/polygon/track_0.6_best_sheep_crop_Passage_brebis_portique_2_30fps.MP4


In [ ]:
frames_generator = sv.get_video_frames_generator(source_path=SOURCE_VIDEO_PATH)
video_info = sv.VideoInfo.from_video_path(video_path=SOURCE_VIDEO_PATH)

setup_zones(video_info.resolution_wh)

with sv.VideoSink(target_path=MAIN_OUTPUT_PATH, 
                  video_info=video_info) as sink:
  heatmap = None
  for i, frame in enumerate(frames_generator):
    annotated_frame = process_frame(frame)

    # to visualize tracking and counting frame by frame
    #sv.plot_image(annotated_frame)

    # Send as frame to video
    sink.write_frame(frame=annotated_frame)


0: 960x1280 2 sheeps, 1021.1ms
Speed: 10.0ms preprocess, 1021.1ms inference, 1733.2ms postprocess per image at shape (1, 3, 960, 1280)
zone presence :  []
unique id set()

0: 960x1280 2 sheeps, 878.6ms
Speed: 15.1ms preprocess, 878.6ms inference, 2.0ms postprocess per image at shape (1, 3, 960, 1280)
zone presence :  []
unique id set()

0: 960x1280 2 sheeps, 902.2ms
Speed: 14.1ms preprocess, 902.2ms inference, 2.0ms postprocess per image at shape (1, 3, 960, 1280)
zone presence :  []
unique id set()

0: 960x1280 4 sheeps, 896.8ms
Speed: 13.0ms preprocess, 896.8ms inference, 2.0ms postprocess per image at shape (1, 3, 960, 1280)
zone presence :  []
unique id set()

0: 960x1280 3 sheeps, 914.6ms
Speed: 14.0ms preprocess, 914.6ms inference, 2.0ms postprocess per image at shape (1, 3, 960, 1280)
zone presence :  []
unique id set()

0: 960x1280 4 sheeps, 979.5ms
Speed: 14.0ms preprocess, 979.5ms inference, 2.0ms postprocess per image at shape (1, 3, 960, 1280)
zone presence :  []
unique id